In [1]:
import random
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation 
from tensorflow.keras.optimizers import RMSprop
import tkinter as tk
from tkinter import scrolledtext
from tensorflow.keras.models import load_model
from googletrans import Translator



In [2]:
text_df=pd.read_csv("fake_or_real_news.csv")

In [3]:
text = list(text_df.text.values)
joined_text="".join(text)

In [4]:
partial_text=joined_text[:10000]

In [5]:
tokenizer=RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [6]:
unique_tokens = np.unique(tokens)
unique_token_index={token: idx for idx, token in enumerate(unique_tokens)}

In [7]:
n_words = 10
input_words =[]
next_words = []
for i in range(len(tokens) - n_words) :
    input_words.append(tokens[i:i + n_words])
    next_words.append(tokens[i +n_words])

In [8]:
 x = np.zeros((len(input_words),n_words ,len(unique_tokens)), dtype =bool)
 y = np.zeros((len(next_words), len(unique_tokens)), dtype =bool)

In [9]:
 for i , words in enumerate(input_words):
     for j ,word in enumerate(words):
         x[i, j , unique_token_index[word]] = 1
         y[i, unique_token_index[next_words[i]]] = 1


In [10]:
model = Sequential()
model.add(LSTM (128 , input_shape=(n_words , len (unique_tokens)), return_sequences = True))
model.add(LSTM (128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [11]:
model.compile(loss="categorical_crossentropy", optimizer =RMSprop(learning_rate=0.01), metrics = ["accuracy"])
model.fit(x , y , batch_size = 128 , epochs =30 , shuffle = True)

Epoch 1/30
14/14 [==============================] - 6s 59ms/step - loss: 6.4364 - accuracy: 0.0354
Epoch 2/30
14/14 [==============================] - 1s 73ms/step - loss: 5.8298 - accuracy: 0.0623
Epoch 3/30
14/14 [==============================] - 1s 60ms/step - loss: 5.7552 - accuracy: 0.0617
Epoch 4/30
14/14 [==============================] - 1s 60ms/step - loss: 5.6390 - accuracy: 0.0658
Epoch 5/30
14/14 [==============================] - 1s 55ms/step - loss: 5.4543 - accuracy: 0.0646
Epoch 6/30
14/14 [==============================] - 1s 59ms/step - loss: 5.1465 - accuracy: 0.0749
Epoch 7/30
14/14 [==============================] - 1s 57ms/step - loss: 4.8201 - accuracy: 0.0955
Epoch 8/30
14/14 [==============================] - 1s 62ms/step - loss: 4.3988 - accuracy: 0.1246
Epoch 9/30
14/14 [==============================] - 1s 55ms/step - loss: 3.9973 - accuracy: 0.1612
Epoch 10/30
14/14 [==============================] - 1s 50ms/step - loss: 3.5914 - accuracy: 0.1978
Epoch 11/

In [12]:
model.save("mymodel.h5")

In [13]:
model = load_model("mymodel.h5")

In [14]:
n_words = 10
def predict_next_word(input_text , n_best):
    input_text = input_text.lower()
    x = np.zeros((1 , n_words , len(unique_tokens)))
    for i , word in enumerate(input_text.split()):
        x[0 , i , unique_token_index[word]] = 1
    predictions = model.predict(x)[0]
    return np.argpartition(predictions , -n_best)[-n_best:]

In [21]:
possible = predict_next_word("he will have to look into this thing and he ", 5)

1/1 [==============================] - 0s 37ms/step


In [22]:
print([unique_tokens[idx] for idx in possible])

['very', 'fire', 'its', 'strategy', 'tape']


In [23]:
def generate_text(input_text ,text_length, creativity = 3):
    word_sequence = input_text.split()
    current=0
    for _ in range(text_length):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence , creativity))]
        except :
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)

In [24]:
generate_text("he will have to look into this thing and he " ,100 ,5)

1/1 [==============================] - 0s 32ms/step


'he will have to look into this thing and he its around what tape how a women in trump strategy trump in to prominent public announced and an will at at have becoming of to months to old everyone t power on the women today else around down when to spectrum will his him him trump hoover have wake investigators after pretend or if or ever way comey hillary an principled comey in its out and comey the old on what an much one in this ever fbi then to limp was the unprecedented after from was s nominee of from an so is has political survival that the a'

In [25]:
# Fonction pour le bouton
def on_button_click():
    input_text = input_entry.get()
    try:
        text_length = int(length_entry.get())
    except ValueError:
        output_text.delete(1.0, tk.END)
        output_text.insert(tk.END, "Please enter a valid number for text length.")
        return
    generated_text = generate_text(input_text, text_length, 5)
    output_text.delete(1.0, tk.END)
    output_text.insert(tk.END, generated_text)

# Création de la fenêtre Tkinter
window = tk.Tk()
window.title("Text Generator")

# Création des widgets
input_label = tk.Label(window, text="Input Text:")
input_label.pack()

input_entry = tk.Entry(window, width=50)
input_entry.pack()

length_label = tk.Label(window, text="Text Length:")
length_label.pack()

length_entry = tk.Entry(window, width=10)
length_entry.pack()

generate_button = tk.Button(window, text="Generate Text", command=on_button_click)
generate_button.pack()

output_label = tk.Label(window, text="Generated Text:")
output_label.pack()

output_text = scrolledtext.ScrolledText(window, width=60, height=20)
output_text.pack()

# Lancement de l'application Tkinter
window.mainloop()

In [26]:
def translate_text(text, dest_language):
    try:
        translated = translator.translate(text, dest=dest_language)
        return translated.text
    except Exception as e:
        return f"Error translating text: {e}"

# Fonction pour le bouton
def on_button_click():
    input_text = input_entry.get()
    try:
        text_length = int(length_entry.get())
    except ValueError:
        output_text.delete(1.0, tk.END)
        output_text.insert(tk.END, "Please enter a valid number for text length.")
        return
    generated_text = generate_text(input_text, text_length, 5)
    translated_text = translate_text(generated_text, lang_entry.get())
    output_text.delete(1.0, tk.END)
    output_text.insert(tk.END, f"Generated Text:\n{generated_text}\n\nTranslated Text:\n{translated_text}")

# Création de la fenêtre Tkinter
window = tk.Tk()
window.title("Text Generator")

# Création des widgets
input_label = tk.Label(window, text="Input Text:")
input_label.pack()

input_entry = tk.Entry(window, width=50)
input_entry.pack()

length_label = tk.Label(window, text="Text Length:")
length_label.pack()

length_entry = tk.Entry(window, width=10)
length_entry.pack()

lang_label = tk.Label(window, text="Language Code (e.g., 'fr' for French, 'ar' for arabe):")
lang_label.pack()

lang_entry = tk.Entry(window, width=10)
lang_entry.pack()

generate_button = tk.Button(window, text="Generate and Translate Text", command=on_button_click)
generate_button.pack()

output_label = tk.Label(window, text="Generated and Translated Text:")
output_label.pack()

output_text = scrolledtext.ScrolledText(window, width=60, height=20)
output_text.pack()

# Lancement de l'application Tkinter
window.mainloop()

1/1 [==============================] - 0s 29ms/step
